In [ ]:
import os
import numpy as np
import scipy.sparse
import scipy.io
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy import stats 
from scipy import * 
import numpy as np
import scanpy as sc
import pandas as pd

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    return adata
batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch"
                 
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster"
                    }  
our_method_emb_dic = {'Immune_Human' : 'Immune_Human_case2_latent_matrix_c_250',
                 'Lung' : 'Lung_case2_latent_matrix_c_300',
                 'Pancreas' : 'Pancreas_case2_latent_matrix_c_250',
                 'Human_Retina': "retina_case2_latent_matrix_c_300",
                 'Human_Mouse' : "Human_Mouse_latent_matrix_c400",
                } 
emb_folder = "/home/krushna/Documents/Data_integration/Ajita_embeddings/checking_other_cases/final/"
subset_list = ["Neutrophil","Erythroid cell","Epithelial cell","Stromal cell","Fetal stromal cell",
               "Macrophage","Endothelial cell","Chondrocyte","B cell","Fasciculata cell","T cell","Dendritic cell",
               "Oligodendrocyte","Monocyte","Smooth muscle cell","Enterocyte","Mast cell","Astrocyte"]

# scDREAMER

In [ ]:
dataset = 'Human_Mouse'
embeddings = our_method_emb_dic[dataset]+'.csv'
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]
adata = load_data(dataset,batch)
emb_path = emb_folder + embeddings 
final_embedings=pd.read_csv(emb_path, sep=',',header=None).values
adata.obsm['final_embeddings'] = final_embedings
adata

In [ ]:
adata = adata[adata.obs[cell_type].isin(subset_list)]
print(adata.obs[cell_type].unique())
print(adata)

In [ ]:
sc.pp.neighbors(adata, use_rep='final_embeddings')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)

In [ ]:
import scIB
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(                
                adata,
                adata,
                batch_key = batch, #need to change according
                label_key = cell_type, #need to change according
                hvg_score_=False,
                cluster_key='cluster',
                cluster_nmi=None,
                ari_=True,
                nmi_=False,
                nmi_method='arithmetic',
                nmi_dir=None,
                silhouette_=True,
                embed= 'final_embeddings', #need to change according 
                si_metric='euclidean',
                pcr_=True,
                cell_cycle_=False,
                organism='mouse',
                isolated_labels_=False,  # backwards compatibility
                isolated_labels_f1_=False,
                isolated_labels_asw_=False,
                n_isolated=None,
                graph_conn_=False,
                kBET_=True,
                kBET_sub=0.5,
                lisi_graph_=True,
                lisi_raw=True,
                trajectory_=False,
                type_=None,
                verbose=True,
        )
results

In [ ]:
import scIB
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(                
                adata,
                adata,
                batch_key = batch, #need to change according
                label_key = cell_type, #need to change according
                hvg_score_=False,
                cluster_key='cluster',
                cluster_nmi=None,
                ari_=False,
                nmi_=False,
                nmi_method='arithmetic',
                nmi_dir=None,
                silhouette_=False,
                embed= 'final_embeddings', #need to change according 
                si_metric='euclidean',
                pcr_=False,
                cell_cycle_=False,
                organism='mouse',
                isolated_labels_=False,  # backwards compatibility
                isolated_labels_f1_=False,
                isolated_labels_asw_=False,
                n_isolated=None,
                graph_conn_=True,
                kBET_=False,
                kBET_sub=0.5,
                lisi_graph_=False,
                lisi_raw=False,
                trajectory_=False,
                type_=None,
                verbose=True,
        )
results

# bbknn

In [ ]:
adata = load_data(dataset,batch)
adata =  adata[adata.obs[cell_type].isin(subset_list)]
adata_int = sc.read_h5ad("/home/krushna/Documents/Data_integration/Othermethods/bbknn/bbknn-Human_Mouse.h5ad")
adata_int = adata_int[adata_int.obs[cell_type].isin(subset_list)]

In [ ]:
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_pca',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_='knn',
        verbose=False,
)
results

# harmony

In [ ]:
adata = sc.read_h5ad('/home/krushna/Documents/Data_integration/Othermethods/harmony/harmony-Human_Mouse.h5ad')
adata = adata[adata.obs[cell_type].isin(subset_list)]

In [ ]:
import scIB 
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =      scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_harmony',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

# insct

In [ ]:
# embeddings = '/home/krushna/Documents/Data_integration/Othermethods/insct/embeddings/Immune_tnn_embeddings_1510.csv'
# adata = load_data(dataset,batch)
# final_embedings=pd.read_csv(embeddings, sep=',',header=None).values
# adata.obsm['final_embeddings'] = final_embedings
# adata = adata[adata.obs[cell_type].isin(subset_list)]

In [ ]:
# import scIB
# results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(                
#                 adata,
#                 adata,
#                 batch_key = batch, #need to change according
#                 label_key = cell_type, #need to change according
#                 hvg_score_=False,
#                 cluster_key='cluster',
#                 cluster_nmi=None,
#                 ari_=True,
#                 nmi_=True,
#                 nmi_method='arithmetic',
#                 nmi_dir=None,
#                 silhouette_=True,
#                 embed= 'final_embeddings', #need to change according 
#                 si_metric='euclidean',
#                 pcr_=False,
#                 cell_cycle_=False,
#                 organism='mouse',
#                 isolated_labels_=False,  # backwards compatibility
#                 isolated_labels_f1_=False,
#                 isolated_labels_asw_=False,
#                 n_isolated=None,
#                 graph_conn_=False,
#                 kBET_=False,
#                 kBET_sub=0.5,
#                 lisi_graph_=False,
#                 lisi_raw=False,
#                 trajectory_=False,
#                 type_=None,
#                 verbose=False,
#         )
# results

# scanorama

In [ ]:
adata = sc.read_h5ad("/home/krushna/Documents/Data_integration/Othermethods/scanorama/scanorama-Human_Mouse.h5ad")
adata = adata[adata.obs[cell_type].isin(subset_list)]
adata

In [ ]:
import scIB
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed='X_emb',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

# scvi

In [ ]:
adata = sc.read_h5ad("/home/krushna/Documents/Data_integration/Othermethods/scvi/scvi-Human_Mouse.h5ad")
adata = adata[adata.obs[cell_type].isin(subset_list)]
adata

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed='X_scVI',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

# seurat

In [ ]:
# adata = load_data(dataset,batch)
# adata = adata[adata.obs[cell_type].isin(subset_list)]
# adata_int = sc.read_h5ad("/home/krushna/Documents/Data_integration/Othermethods/seurat/seurat-Immune_Human.h5ad")
# adata_int = adata_int[adata_int.obs[cell_type].isin(subset_list)]
# adata_int

In [ ]:
# import scIB
# #Trajectory is asking precomputed sudo time point
# results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(
#         adata,
#         adata_int,
#         batch_key = batch,
#         label_key = cell_type,
#         hvg_score_=False,
#         cluster_key='cluster',
#         cluster_nmi=None,
#         ari_=True,
#         nmi_=True,
#         nmi_method='arithmetic',
#         nmi_dir=None,
#         silhouette_=True,
#         embed= 'X_pca',
#         si_metric='euclidean',
#         pcr_=False,
#         cell_cycle_=False,
#         organism='mouse',
#         isolated_labels_=False,  # backwards compatibility
#         isolated_labels_f1_=False,
#         isolated_labels_asw_=False,
#         n_isolated=None,
#         graph_conn_=False,
#         kBET_=False,
#         kBET_sub=0.5,
#         lisi_graph_=False,
#         lisi_raw=False,
#         trajectory_=False,
#         type_=None,
#         verbose=False,
# )
# results